In [1]:
import pandas as pd
import numpy as np

import plotly.plotly as py

## Objective
This notebook is meant to test a proposed scoring system for Fantasy Label, using historical Billboard data downloaded [from this site](http://mikekling.com/analyzing-the-billboard-hot-100/). 

Eventually, we'll want to test (and ultimately use) this algorithm on the [live Billboard RSS feed](https://www.billboard.com/rss/charts/hot-100). We'll likely want to use [feedparser](http://localhost:8888/notebooks/FL%20Exploratory.ipynb#Scores-Over-Time) for that.

## Collect & Prepare Data

In [2]:
#data downloaded from http://mikekling.com/analyzing-the-billboard-hot-100/
data = open("./data/all_billboard_data.txt", encoding='utf-8', errors='ignore').readlines()

#remove some lines that are improperly encoded and write to a new "cleaned" csv
cleaned_data = open("./data/cleaned_billboard_data.txt","w")
for line in data:
    cleaned_data.write(line)

In [3]:
df = pd.read_csv("./data/cleaned_billboard_data.txt", sep = "|")

In [4]:
df.columns = ['pos',
              'last_week',
              'peak',
              'weeks_on_chart',
              'title',
              'artist',
              'chart_entry_date',
              'entry_position',
              'overall_peak',
              'overall_weeks_on_chart',
              'chart_date']

In [5]:
df.head()

,pos,last_week,peak,weeks_on_chart,title,artist,chart_entry_date,entry_position,overall_peak,overall_weeks_on_chart,chart_date
0,1,1,1.0,15.0,Uptown Funk!,MARK RONSON featuring BRUNO MARS,41972.0,65.0,1.0,15.0,20150307
1,2,2,2.0,20.0,Thinking Out Loud,ED SHEERAN,41937.0,69.0,2.0,20.0,20150307
2,3,6,3.0,7.0,Love Me Like You Do,ELLIE GOULDING,42028.0,45.0,3.0,7.0,20150307
3,4,5,4.0,6.0,Sugar,MAROON 5,42035.0,8.0,4.0,6.0,20150307
4,5,3,2.0,28.0,Take Me To Church,HOZIER,41881.0,96.0,2.0,28.0,20150307


In [6]:
print("Historical data covers data from {0} to {1}.".format(df.chart_date.min(), df.chart_date.max()))

Historical data covers data from 19400803 to 20150307.


Take subset of original data that we need 

In [7]:
cols = ['pos','last_week','title','chart_date','artist']

In [8]:
df_clean = df[cols]
df_clean.head()

,pos,last_week,title,chart_date,artist
0,1,1,Uptown Funk!,20150307,MARK RONSON featuring BRUNO MARS
1,2,2,Thinking Out Loud,20150307,ED SHEERAN
2,3,6,Love Me Like You Do,20150307,ELLIE GOULDING
3,4,5,Sugar,20150307,MAROON 5
4,5,3,Take Me To Church,20150307,HOZIER


## Handling Multiple Artists
For tracks with multiple artists, we need to duplicate rows in our dataset so that each artist gets full credit.

In [9]:
def split_artist(artists):
    stop_words = ['featuring','&', ',']
    for word in stop_words:
        artists = artists.replace(word, '|')
    
    a_list = [s.strip() for s in artists.split('|')]
    return a_list

In [10]:
df_artist_list = df_clean.assign(artist_list = [split_artist(a) for a in df.artist])

In [11]:
df_artist_list.head()

,pos,last_week,title,chart_date,artist,artist_list
0,1,1,Uptown Funk!,20150307,MARK RONSON featuring BRUNO MARS,"[MARK RONSON, BRUNO MARS]"
1,2,2,Thinking Out Loud,20150307,ED SHEERAN,[ED SHEERAN]
2,3,6,Love Me Like You Do,20150307,ELLIE GOULDING,[ELLIE GOULDING]
3,4,5,Sugar,20150307,MAROON 5,[MAROON 5]
4,5,3,Take Me To Church,20150307,HOZIER,[HOZIER]


In [12]:
#adapted from https://stackoverflow.com/questions/32468402/how-to-explode-a-list-inside-a-dataframe-cell-into-separate-rows/32470490
#for each row, append it X times to the rows list depending on the # of main artists

rows = []
_ = df_artist_list.apply(
    lambda row: [rows.append([row.pos, row.last_week, row.title, row.chart_date, artist]) for artist in row.artist_list], 
    axis=1
    )

In [13]:
df_new = pd.DataFrame(rows, columns=cols)
df_flat = df_new.set_index(pd.to_datetime(df_new.chart_date.astype(str))).to_period(freq='7D')
df_flat.head()

,pos,last_week,title,chart_date,artist
chart_date,,,,,
2015-03-07,1,1,Uptown Funk!,20150307,MARK RONSON
2015-03-07,1,1,Uptown Funk!,20150307,BRUNO MARS
2015-03-07,2,2,Thinking Out Loud,20150307,ED SHEERAN
2015-03-07,3,6,Love Me Like You Do,20150307,ELLIE GOULDING
2015-03-07,4,5,Sugar,20150307,MAROON 5


## Scoring System
At a high-level, the initial scoring system scores tracks according to two dimensions: **this week's billboard rank** and **change in billboard rank from last week**.

$trackScore = (101 - thisWeekRank) * [1 +  0.01(lastWeekRank - thisWeekRank)]$

The multiplier in the 2nd term is meant to reward highly *volatile* tracks i.e. tracks that have jumped high in the chart. For example, in the most extreme case, where a track is not on the chart but enters at #1, the volatility multiplier is 2X.

In [14]:
#score calculators
def score_track(row):
    last_week_rank = row['last_week'] 
    this_week_rank = row['pos']
    
    if last_week_rank != 'NEW':
        volatility = int(last_week_rank) - this_week_rank
    else:
        volatility = 101 - this_week_rank

    score = (101-this_week_rank) * (1 + 0.01 * volatility)
    return score

To aggregate track scores on the artist level, we apply an exponentially decaying weighted sum across all of that artist's tracks on the chart. This is to help correct for artists releasing big albums that take up many slots on the chart.

In [15]:
def score_track_list(track_list):
    #list of track_scores
    weights = [1/2**x for x in range(len(track_list))]
    sorted_list = sorted(track_list, reverse=True)
    
    return sum([w*t for w,t in zip(weights, sorted_list)])
    

In [16]:
df_flat['track_score'] = df_flat.apply(score_track, axis=1)

In [17]:
df_flat.head(10)

,pos,last_week,title,chart_date,artist,track_score
chart_date,,,,,,
2015-03-07,1,1,Uptown Funk!,20150307,MARK RONSON,100.00
2015-03-07,1,1,Uptown Funk!,20150307,BRUNO MARS,100.00
2015-03-07,2,2,Thinking Out Loud,20150307,ED SHEERAN,99.00
2015-03-07,3,6,Love Me Like You Do,20150307,ELLIE GOULDING,100.94
2015-03-07,4,5,Sugar,20150307,MAROON 5,97.97
2015-03-07,5,3,Take Me To Church,20150307,HOZIER,94.08
2015-03-07,6,4,FourFiveSeconds,20150307,RIHANNA,93.10
2015-03-07,6,4,FourFiveSeconds,20150307,KANYE WEST,93.10
2015-03-07,6,4,FourFiveSeconds,20150307,PAUL McCARTNEY,93.10


This query returns the top artists according our scoring system applied to the 20150307 chart

In [18]:
df_flat.loc['20150307'].groupby('artist')['track_score'].agg(score_track_list).sort_values(ascending=False).head(10)

artist
TAYLOR SWIFT      172.735000
DRAKE             172.436460
NICKI MINAJ       153.244375
ARIANA GRANDE     144.877500
CHRIS BROWN       137.435000
LIL WAYNE         136.522500
THE WEEKND        134.080000
NE-YO             132.090000
MEGHAN TRAINOR    130.270000
ELLIE GOULDING    129.650000
Name: track_score, dtype: float64

This query returns the top artists without applying the weighted sum. This demonstrates how the weighted sum only affects relative ranking slightly, but has great effect on the raw score.

In [19]:
df_flat.loc['20150307'].groupby('artist')['track_score'].sum().sort_values(ascending=False).head(10)

artist
DRAKE             599.06
TAYLOR SWIFT      349.86
NICKI MINAJ       296.67
ARIANA GRANDE     222.13
CHRIS BROWN       217.40
LIL WAYNE         193.94
THE WEEKND        188.12
SAM SMITH         184.09
NE-YO             172.38
MEGHAN TRAINOR    170.45
Name: track_score, dtype: float64

## Scores Over Time
This analysis helps us to understand the volatility of scores week-to-week. Ideally, we want just enough volatility/unpredictability such that the results of each week are surprising enough to the user to want to come back.

In [20]:
artist_list = ['DRAKE','TAYLOR SWIFT','NICKI MINAJ','ARIANA GRANDE','CHRIS BROWN']
artist_list2 = ['LIL WAYNE','THE WEEKND','SAM SMITH','NE-YO', 'MEGHAN TRAINOR']

In [21]:
#groupby artist and calculate agg score across tracks
artist_series = df_flat.groupby([df_flat.index.get_level_values(0), 'artist'])['track_score'].agg(score_track_list)

#reshape artist df such that each column is a different artist, indexed by weekly period for easier slicing and plotting
df_artist = artist_series.to_frame().reset_index(1).pivot(columns='artist', values='track_score')

In [22]:
#using plotly/cufflinks for visualization: https://plot.ly/pandas/line-charts/#plot-entire-dataframe-in-cufflinks
import cufflinks as cf
cf.set_config_file(offline=False, world_readable=True, theme='ggplot')

In [23]:
df_artist.loc['2014-01-01':][artist_list].iplot(kind='scatter')